In [2]:
import xml.sax
import spacy
from nltk.stem import *
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import sys
from helper import *
import math
import copy
import Stemmer
import timeit
import os 
import heapq
import copy
from os import listdir
from os.path import isfile, join

stemming = Stemmer.Stemmer('english')
stop_words = set(stopwords.words('english'))

In [3]:
import re
r1 = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',re.DOTALL)
r2 = re.compile(r'{\|(.*?)\|}',re.DOTALL)
r3 = re.compile(r'{{v?cite(.*?)}}',re.DOTALL)
r4 = re.compile(r'[-.,:;_?()"/\']',re.DOTALL)
r5 = re.compile(r'\[\[file:(.*?)\]\]',re.DOTALL)
catRegExp = r'\[\[category:(.*?)\]\]'
r6 = re.compile(r'[\'~` \n\"_!=@#$%\-^*+{\[}\]\|\\<>/?]',re.DOTALL)
r7 = re.compile(r'{{infobox(.*?)}}',re.DOTALL)
r9 = re.compile(r'{{(.*?)}}',re.DOTALL)
r10 = re.compile(r'<(.*?)>',re.DOTALL)
r11=re.compile(r'\[\[(.*?)\]\]')
r12=re.compile(catRegExp,re.DOTALL)

In [4]:
import xml.sax
import subprocess
import mwparserfromhell
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


inverted_index = {}
no_of_words=0
file_no=0
word_count_in_page_no={}
total_docs=30000
total_titles=30000
total_words=40000
page_count=0

_page_id=0
title_file_no=0
list_for_title=[]

In [5]:
class WikiXmlHandler(xml.sax.handler.ContentHandler):
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._pages = []
#         self._page_id=0

    def characters(self, content):
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        if name in ('title', 'text'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)
            
        if name == 'title':

            create_title(' '.join(self._buffer))

        if name == 'page':
            
            self._pages.append((self._values['title'], self._values['text']))
            create(self._pages[0])
            self._pages=[]
            

In [6]:
def field_idf_for_word(doc_list):
    total_doc=8796395
    t_ct=1
    c_ct=1
    i_ct=1
    r_ct=1
    e_ct=1
    b_ct=1
    for doc in doc_list:
        posting_t=re.findall(r't[0-9]*',doc,re.DOTALL)
        posting_c=re.findall(r'c[0-9]*',doc,re.DOTALL)
        posting_i=re.findall(r'i[0-9]+',doc,re.DOTALL)   #otherwise it might match with id123
        posting_r=re.findall(r'r[0-9]*',doc,re.DOTALL)
        posting_e=re.findall(r'e[0-9]*',doc,re.DOTALL)
        posting_b=re.findall(r'b[0-9]*',doc,re.DOTALL)
        
        if(len(posting_t)):
            t_ct+=1
        if(len(posting_c)):
            c_ct+=1
        if(len(posting_i)):
            i_ct+=1
        if(len(posting_r)):
            r_ct+=1
        if(len(posting_e)):
            e_ct+=1
        if(len(posting_b)):
            b_ct+=1
    
    idf_t=total_doc/t_ct
    idf_t=round(math.log(1+idf_t,10),3)
    
    idf_c=total_doc/c_ct
    idf_c=round(math.log(1+idf_c,10),3)
    
    idf_i=total_doc/i_ct
    idf_i=round(math.log(1+idf_i,10),3)
    
    idf_r=total_doc/r_ct
    idf_r=round(math.log(1+idf_r,10),3)
    
    idf_e=total_doc/e_ct
    idf_e=round(math.log(1+idf_e,10),3)
    
    idf_b=total_doc/b_ct
    idf_b=round(math.log(1+idf_b,10),3)
    
    idf_res=[idf_t,idf_c,idf_i,idf_r,idf_e,idf_b]
    
    return idf_res


def tfidf_posting(tf_posting,field_idf_list):
    tfidf_pos=""
#     print(tf_posting)
#     print(field_idf_list)
    
    
    docid=re.findall(r'D[0-9]*',tf_posting,re.DOTALL)
#     docid=int(re.findall(r'[0-9]+',docid[0],re.DOTALL)[0])
    tfidf_pos+=docid[0]       # doc id added
    
    t_pos=re.findall(r't[0-9]*',tf_posting,re.DOTALL)
    if(len(t_pos)):
        t_tf=int(re.findall(r'[0-9]+',t_pos[0],re.DOTALL)[0])
        tfidf=int(math.floor(t_tf*field_idf_list[0]))
        tfidf_pos+="t"+str(tfidf)
        
    c_pos=re.findall(r'c[0-9]*',tf_posting,re.DOTALL)
    if(len(c_pos)):
        c_tf=int(re.findall(r'[0-9]+',c_pos[0],re.DOTALL)[0])
        tfidf=int(math.floor(c_tf*field_idf_list[1]))
        tfidf_pos+="c"+str(tfidf)
        
    i_pos=re.findall(r'i[0-9]*',tf_posting,re.DOTALL)
    if(len(i_pos)):
        i_tf=int(re.findall(r'[0-9]+',i_pos[0],re.DOTALL)[0])
        tfidf=int(math.floor(i_tf*field_idf_list[2]))
        tfidf_pos+="i"+str(tfidf)
        
    r_pos=re.findall(r'r[0-9]*',tf_posting,re.DOTALL)
    if(len(r_pos)):
        r_tf=int(re.findall(r'[0-9]+',r_pos[0],re.DOTALL)[0])
        tfidf=int(math.floor(r_tf*field_idf_list[3]))
        tfidf_pos+="r"+str(tfidf)
    
    e_pos=re.findall(r'e[0-9]*',tf_posting,re.DOTALL)
    if(len(e_pos)):
        e_tf=int(re.findall(r'[0-9]+',e_pos[0],re.DOTALL)[0])
        tfidf=int(math.floor(e_tf*field_idf_list[4]))
        tfidf_pos+="e"+str(tfidf)
    
    b_pos=re.findall(r'b[0-9]*',tf_posting,re.DOTALL)
    if(len(b_pos)):
        b_tf=int(re.findall(r'[0-9]+',b_pos[0],re.DOTALL)[0])
        tfidf=int(math.floor(b_tf*field_idf_list[5]))
        tfidf_pos+="b"+str(tfidf)
        
        
    return tfidf_pos

In [7]:
def dump_title():
    global title_file_no
    global list_for_title
    print("...................title......................")
    title_pointer=open("dumped_titles"+"/"+str(title_file_no)+".txt",'w')
    title_file_no+=1
    for each in list_for_title:
        title_pointer.write(each)
    list_for_title=[]
def create_title(title_content):
#     print(title_content)
    global title_file_no
    global list_for_title
    global total_titles
    title_content+="\n"
    list_for_title.append(title_content)
#     print(list_for_title)
    if(len(list_for_title)>=total_titles):
        dump_title()

In [8]:
# def find_idf(last_record):
#     total_pages_allowed=8796395
#     doc_count = last_record[1].split("#")
#     idf = total_pages_allowed/len(doc_count)
#     idf = math.log(1 + idf, 10)
#     idf=round(idf,3)
#     print(idf)
#     print(doc_count)
#     return idf,doc_count

def merge_function(path_to_doc):
#     print("maaddhhaaann")
    global total_words
    count_of_files=0
    count_of_words=0
    
    all_files=os.listdir(path_to_doc)
    sz=len(all_files)
    print(sz)
    
    material=os.listdir()
    if("destination" not in material):
        os.mkdir("destination")
    
    primary_index=open("."+"/"+"destination"+"/"+str(count_of_files)+".txt","w")
    seconday_index=open("secondary.txt","w")
    
    list_of_file_ptrs=[]
    for loc in range(sz):
        pointers=open("."+"/"+"dumped_files"+"/"+"doc"+str(loc)+".txt","r")
        list_of_file_ptrs.append(pointers)    
    #remmember these are from 1 and files are created with name 0 
    list_for_heap=[]
    for loc in range(sz):
        read=list_of_file_ptrs[loc].readline()
#         print(read)
#         print(".........................................................")
        find_partition=read.index("#")
#         print(find_partition)
        vocab=read[:find_partition]
        posting=read[find_partition+1:]
        posting=posting[:len(posting)-1]
        fin_lis=[]
        fin_lis.append(vocab)
        fin_lis.append(posting)
        fin_lis.append(loc)
        list_for_heap.append(fin_lis)
    heapq.heapify(list_for_heap)
    
    final_record=heapq.heappop(list_for_heap)     # first minimum popped out
    last_record=copy.deepcopy(final_record)
    

    while(len(list_for_heap)>0):
#         print(size_of_heap)
        rd=list_of_file_ptrs[last_record[2]].readline()
        if(rd !=""):
#             print(rd)
            find_part=rd.index("#")
            voc=rd[:find_part]
            post=rd[find_part+1:]
            post=post[:len(post)-1]
            nw_lis=[]
            nw_lis.append(voc)
            nw_lis.append(post)
            nw_lis.append(last_record[2])
            heapq.heappush(list_for_heap,nw_lis)
        buffer_record=heapq.heappop(list_for_heap)           # deleting another node to check weather this belong to same
                                                             # word in different document or not 
        
        if(buffer_record[0]!=last_record[0]):
            
            docs=last_record[1].split("#")
            

#             idf = total_docs/len(docs)              #overall idf         not needed
#             idf = round(math.log(1 + idf, 10), 3)
            
            line_to_write=last_record[0]                   #word to be written 
            
            field_idf_list=field_idf_for_word(docs)   # get field tfidf list here index 0=t,1=c,2=i,3=r,4=e,5=b
            

    
            
            for doc in docs:                        #modifying each doc's field tf to tfidf and writing to merged file
                tf_posting_section=doc
                if(len(tf_posting_section)>0):
                    id_=re.findall('D',tf_posting_section,re.DOTALL)
                    if(len(id_)):
                        tfidf_posting_section= tfidf_posting(tf_posting_section,field_idf_list) #convert tf posting  part to tfidf posting part
    
                        line_to_write+="#"+tfidf_posting_section #v.1
                  
            if(line_to_write!=last_record[0]):
                primary_index.write(line_to_write+'\n')
                
            if count_of_words==0:   #first time entry after write happpen so write the first word into secondary index
                print("sec")
                seconday_index.write(last_record[0] + "\n")  
            count_of_words+=1    
            if count_of_words==total_words:

                count_of_words=0
                count_of_files+=1
                primary_index=open("."+"/"+"destination"+"/"+str(count_of_files)+".txt","w")
                
            last_record=buffer_record 
        else:
            last_record[2]=buffer_record[2]
            last_record[1]=last_record[1]+"#"+buffer_record[1]
    print("merge complete")

In [11]:
i=0;
temp=os.listdir()
if "dumped_files" not in temp:
    os.mkdir("dumped_files")
if "dumped_titles" not in temp:
    os.mkdir("dumped_titles")
data_path="data/"
TOTAL_DATA=os.listdir(data_path)

for data in TOTAL_DATA:
#     if(i>=1):
#         break
    handler = WikiXmlHandler()
    parser = xml.sax.make_parser()
    parser.setContentHandler(handler)
    print(data)
#     i+=1
#     if data == "enwiki-20200801-pages-articles-multistream23.xml-p29823661p30503450.bz2":
#         continue
    for line in subprocess.Popen(['bzcat'], 
                              stdin = open(data_path+data), 
                              stdout = subprocess.PIPE).stdout:
        parser.feed(line)
    
#     dump_title()

    if(len(inverted_index)>0):
        dump_title()
        start_dump()
    print("documents processed",_page_id)
print("enter merge")
merge_function("."+"/"+"dumped_files")

enwiki-20200801-pages-articles-multistream23.xml-p29823661p30503450.bz2
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
documents processed 211825
enwiki-20200801-pages-articles-multistream6.xml-p565314p892912.bz2
...................title......................
....................................
...................title......................
...............................

...................title......................
....................................
documents processed 2531634
enwiki-20200801-pages-articles-multistream23.xml-p26823661p28323660.bz2
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
...................title......................
.............

...................title......................
....................................
documents processed 5132380
enwiki-20200801-pages-articles-multistream5.xml-p352690p565313.bz2
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
documents processed 5253246
enwiki-20200801-pages-articles-multistream1.xml-p1p30303.bz2
...................title......................
....................................
documents processed 5273043
enwiki-20200801-pages-articles-multistream17.xml-p13039267p13693073.bz2
...................title......................
....................................
...................title....................

...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
documents processed 7617885
enwiki-20200801-pages-articles-multistream21.xml-p22722157p23927983.bz2
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
...................title......................
....................................
...................title......................
.............

IndexError: list index out of range

In [12]:
merge_function("."+"/"+"dumped_files")

346
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec
sec


In [9]:
def validate(word):
    try:
        word.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

def clean(tokenList):
    tokenList = ' '.join(tokenList)
    tokenList = r4.sub(' ',tokenList)
    tokenList = r5.sub(' ',tokenList)
    tokenList = r6.sub(' ',tokenList)
    tokenList = tokenList.split()
    return tokenList

def isValid(word):
    if word == "" or word in stop_words or len(word) < 3:
            return False
    return validate(word)


def process_digit(w):
    if w.isnumeric():
        w=w.lstrip('0')
    return w  




def find_tf_for_field(TF,fraction):
    TF=int((fraction+TF)*10000)
    return TF



def add_in_inverted_index(dict_,tag,page_id):
    word_ctr_in_dict=0
    for i in dict_:
        word_ctr_in_dict+=dict_[i]
    for i in dict_:
        if i not in inverted_index:
            inverted_index[i] = {}
        if page_id not in inverted_index[i]:
            inverted_index[i][page_id] = {}
#         inverted_index[i][page_id][tag] = dict_[i]
            
        TF=dict_[i]/word_ctr_in_dict          #find tf corresponding to every tag/field
        TF=round(math.log(1+TF,10),4)
        
        if tag=="title":
            TF=find_tf_for_field(TF,.0100)       #priorities are assigned accordingly
        elif tag=="category":
            TF=find_tf_for_field(TF,.0050)
        elif tag=="infobox":
            TF=find_tf_for_field(TF,.0050)
        elif tag=="external_links":
            TF=find_tf_for_field(TF,.0010)
        elif tag=="references":
#             print(TF,"r")
            TF=find_tf_for_field(TF,.0010)
        elif tag=="body":
            TF=find_tf_for_field(TF,.0001)
        inverted_index[i][page_id][tag]=[dict_[i],TF]
                                         

In [10]:
# def find_tf(sentence,wrd_ctr,word_count_in_page_no,each_id):
#     term_factor = wrd_ctr/word_count_in_page_no[each_id]
#     term_factor = math.log(1 + term_factor, 10)
# #     print(term_factor)
#     term_factor = round(term_factor, 4)
# #     print(term_factor)
#     term_factor = int(((0.0001)+term_factor)*10000)
# #     print(term_factor)
#     sentence += "*" + str(term_factor)
#     return sentence


def start_dump():
    print("....................................")
    global inverted_index
    global no_of_words
    global file_no
    global page_count
    dump_inverted_index()
    inverted_index = {}
    page_count=0
    file_no+=1
    word_count_in_page_no={}
def dump_inverted_index(): 
    global file_no
    global word_count_in_page_no
    indices=inverted_index.keys()
    indices=sorted(indices)
    f= open("."+"/"+"dumped_files"+"/"+"doc"+str(file_no)+".txt","w")
    for val in indices:
#         wrd_ctr=0
        sentence=""
        sentence+=str(val)
        further_info=inverted_index[val]
        document_ids=further_info.keys()
        document_ids=sorted(document_ids)
        for each_id in document_ids:
            
            sentence+="#"+"D"+ str(each_id)
            inverted_index[val][each_id].keys()
#             print(inverted_index[val][each_id])
#             print(list(inverted_index[val][each_id].keys())[0])
#             print(list(inverted_index[val][each_id].keys()))
            if("title" in list(inverted_index[val][each_id].keys())):
                sentence+="t"+str(inverted_index[val][each_id]["title"][1])
#                 print("t")
#                 wrd_ctr+=inverted_index[val][each_id]["title"][0]
            if("body" in list(inverted_index[val][each_id].keys())):
                sentence+="b"+str(inverted_index[val][each_id]["body"][1])
#                 wrd_ctr+=inverted_index[val][each_id]["body"][0]
#                 print("b")
            if("infobox" in list(inverted_index[val][each_id].keys())):
                sentence+="i"+str(inverted_index[val][each_id]["infobox"][1])
#                 wrd_ctr+=inverted_index[val][each_id]["infobox"][0]
#                 print("i")
            if("category" in list(inverted_index[val][each_id].keys())):
                sentence+="c"+str(inverted_index[val][each_id]["category"][1])
#                 wrd_ctr+=inverted_index[val][each_id]["category"][0]
#                 print("c")
            if("external_links" in list(inverted_index[val][each_id].keys())):
                sentence+="e"+str(inverted_index[val][each_id]["external_links"][1])
#                 wrd_ctr+=inverted_index[val][each_id]["external_links"][0]
#                 print("e")
            if("references" in list(inverted_index[val][each_id].keys())):
                sentence+="r"+str(inverted_index[val][each_id]["references"][1])
#                 wrd_ctr+=inverted_index[val][each_id]["references"][0]
#                 print("r")
#             print(each_id)
#             sentence=find_tf(sentence,wrd_ctr,word_count_in_page_no,each_id)
            
            
        f.write(sentence + "\n")
    f.close()

In [11]:
def initial_preprocess(tex):
    tex = tex.lower()
    tex = r1.sub(' ', tex)
    tex = r2.sub(' ', tex)
    tex = r10.sub(' ', tex)
    tex.replace('\n', ' ')
    tex= tex.strip()
    return tex

def intermediate_preprocess(tex):
    tex = r3.sub(' ', tex)
    return tex


def fin_preprocess(tex):
    tex = r4.sub(' ', tex)
    tex = r5.sub(' ', tex)
    tex = r6.sub(' ', tex)
    tex = r7.sub(' ', tex)
    tex = r9.sub(' ', tex)
    tex = r11.sub(' ', tex)
    tex = re.compile(r'\[\[category:(.*?)\]\]',re.DOTALL).sub(' ', tex)
    tex = tex.split()
    return tex

def create(pge):
    global page_count
    global _page_id
    page_count+=1
#     print(page_count)
    global word_count_in_page_no
    word_count_for_page=0
    Title_dict={}
    Body_Text_dict={}
    Infobox_dict={}
    Categories_dict={}
    External_Links_dict={}
    References_dict={}
    
    page_id=_page_id
    title=pge[0]
    body=pge[1]
    title=title.lower()
    
    body=initial_preprocess(body)
    
    
    references=re.findall(r'{{v?cite(.*?)}}',body,re.DOTALL)


    for i in references:
        refer=re.findall(r'\| ?title ?=(.*?)\|',i,re.DOTALL) 
        str_refer=""
        str_refer=str_refer.join(refer)
        ref_tok=str_refer.split()
        token_list=clean(ref_tok)
        token_list=stemming.stemWords(token_list)
        for token in token_list:
            word=token
            if isValid(word):
                word_count_for_page+=1
                if word not in References_dict:
                    References_dict[word]=1
                else:
                    References_dict[word]+=1

        
    body=intermediate_preprocess(body)
    
    
    
    linkcontent=re.findall(r'== ?external links ?==.*',body,re.DOTALL) 

    if len(linkcontent)!=0:
        linkcontent=r11.sub(' ',linkcontent[0])
        ll=re.findall(r'\[(.*?)\]', linkcontent, flags=re.MULTILINE)
        link=clean(ll)
        link=stemming.stemWords(link)
        for l in link:
            word=l
            if isValid(word):
                word_count_for_page+=1
                if l not in External_Links_dict:
                    External_Links_dict[l]=1
                else:
                    External_Links_dict[l]+=1
    
    
    categories = re.findall(r'\[\[category:(.*?)\]\]', body, flags=re.MULTILINE)
    
    for category in categories:
        tt = category.split()
        token_list = clean(tt)
        token_list=stemming.stemWords(token_list)
        for token in token_list:
            word=token
#             word = stemmer.stem(token)
            word.replace('\n', ' ')
            word= word.strip()
#             word = remove_new_line(word)
            if isValid(word):
                word_count_for_page+=1
                if word not in Categories_dict:
                    Categories_dict[word] = 1
                else:
                    Categories_dict[word] += 1
    
    
    
    title = title.split()
    title=clean(title)
    title=stemming.stemWords(title)
    for i in title:  
#         word = stemmer.stem(i)  
        word=i
        if isValid(word):
            word_count_for_page+=1
            if word not in Title_dict:
                Title_dict[word] = 1
            else:
                Title_dict[word] += 1
    
    
    



    infobox = re.findall(r'{{infobox(.*?)}}', body, re.DOTALL)
    str_info=""
    str_info=str_info.join(infobox)
    
    pp=re.findall('^(.*?)\|',str_info)
    if(len(pp)>0):
        Infobox_dict[pp[0].strip()] = 1
    for infoList in infobox:
        tt = re.findall(r'=(.*?)\|',infoList,re.DOTALL)
        token_list = clean(tt)
        token_list=stemming.stemWords(token_list)
        for token in token_list:
#                 word = stemmer.stem(token)
                word=token
                word.replace('\n', ' ')
                word= word.strip()
#                 word = remove_new_line(word)
                
                if isValid(word):
                        word_count_for_page+=1
                        if word not in Infobox_dict:
                            Infobox_dict[word] = 1
                        else:
                            Infobox_dict[word] += 1
        
    
    body=fin_preprocess(body)
    fin_body=clean(body)

    fin_body=stemming.stemWords(fin_body)
    for token in fin_body:
        word=token
#         word = stemmer.stem(token)
        word.replace('\n', ' ')
        word= word.strip()
        if isValid(word):
            word_count_for_page+=1
            if word not in Body_Text_dict:
                Body_Text_dict[word] = 1
            else:
                Body_Text_dict[word] += 1
    
    word_count_in_page_no[page_id]=word_count_for_page
#     print(word_count_in_page_no)
    
    add_in_inverted_index(Title_dict, "title", page_id)
    add_in_inverted_index(Body_Text_dict, "body", page_id)
    add_in_inverted_index(Infobox_dict, "infobox", page_id)
    add_in_inverted_index(Categories_dict, "category", page_id)
    add_in_inverted_index(External_Links_dict, "external_links", page_id)
    add_in_inverted_index(References_dict, "references", page_id)
    

    if(page_count == total_docs):
#         print("happy")
        start_dump()
    _page_id+=1
    
    
    
    
    


In [1]:
l={1:"e",2:"j"}
for i in l:
    print(l[i])

e
j


In [12]:
l=os.listdir("dumped_files")
l=sorted(l)
len(l)

346